# Sprawozdanie Lab2



1. Założenia – jak duże liczby pierwsze mogą być wykorzystane w programie? 

Dlaczego liczby peirwsze?
Liczby pierwsze mają unikalne właściwości matematyczne, które są wykorzystywane w obliczeniach funkcji Eulera φ(n)

RSA opiera się na trudności faktoryzacji dużych liczb całkowitych. Moduł n jest iloczynem dwóch dużych liczb pierwszych p i q.
Znalezienie p i q na podstawie n (czyli rozkład n na czynniki pierwsze) jest problemem trudnym obliczeniowo, szczególnie dla dużych liczb pierwszych. To właśnie ta trudność zapewnia bezpieczeństwo RSA


Jednocześnie n (moduł kluczy obliczany z n = p * q ) musi być większy niż rozmiar wiadomości, którą szyfrujemy. 
N służy jako modulo, co obetnie nam część wiadomości i nie będziemy w stanie jej odszyfrować.
Dla 4-cyfrowego p i q, maksymalny rozmiar wiadomości może mieć 2-3 znaki



---
# 2. Opis metod użytych do wyznaczania e i d. 

## e (eksponent klucza publicznego)
### Założenia:

e musi być liczbą całkowitą spełniającą warunki:
1 < e < φ(n) (gdzie φ(n) to funkcja Eulera dla n).
gcd(e, φ(n)) = 1 (czyli e i φ(n) muszą być względnie pierwsze).
Dodatkowo, w implementacji sprawdzamy, czy e jest liczbą pierwszą, co zwiększa bezpieczeństwo.

### Metoda:

Generujemy losową liczbę e w zakresie od 2 do φ(n) - 1.
Sprawdzamy, czy gcd(e, φ(n)) == 1 (za pomocą algorytmu Euklidesa).
Jeśli warunek jest spełniony, przyjmujemy tę wartość jako e.
Kod:

def generate_e(phi):
    while True:
        e = random.randint(2, phi - 1)
        if gcd(e, phi) == 1 and isprime(e):
            return e




## d (eksponent klucza prywatnego) jest wyznaczany jako odwrotność e modulo (p-1)(q-1).
### Założenia:

d musi być odwrotnością modularną e względem φ(n), czyli (d * e) % φ(n) = 1

### Metoda:

Wykorzystujemy rozszerzony algorytm Euklidesa do obliczenia odwrotności modularnej.
def generate_d(e, phi):
    d = pow(e, -1, phi)
    return d




3. Opis realizacji zadań (programu i jego składowych) i wartości uzyskane podczas ich 
realizacji. 


In [1]:
import random 

from sympy import isprime 

from math import gcd 

def generate_e(phi): 

    while True: 
        e = random.randint(2, phi - 1) 

        # Sprawdź, czy gcd(e, phi) == 1 i czy e jest liczbą pierwszą 

        if gcd(e, phi) == 1 and isprime(e): 

            return e 

def generate_d(e, phi): 

    d = pow(e, -1, phi) 

    return d 


def generate_keys(p, q): 

    """ 

    Generuje klucz publiczny i klucz prywatny. 

    :return: klucz publiczny i klucz prywatny. 

    """ 

    # Obliczanie modułu n i funkcji Eulera phi 

    n = p * q 

    phi = (p - 1) * (q - 1) 

 
 

    # Generowanie eksponenta klucza publicznego e 

    e = generate_e(phi) 

 
 

    # Generowanie eksponenta klucza prywatnego d 

    d = generate_d(e, phi) 

 
 

    # Klucz publiczny: (e, n), klucz prywatny: (d, n) 

    public_key = (e, n) 

    private_key = (d, n) 

 
 

    return {"public_key": public_key, "private_key": private_key} 


def generate_prime(min, max): 

    while True: 

        # Generuj losową 4-cyfrową liczbę 

        num = random.randint(min, max) 

        # Sprawdź, czy liczba jest pierwsza 

        if isprime(num): 

            return num 

In [2]:


def encrypt_message(m, e, n): 

    """ 

    Szyfrowanie wiadomości. 

    :param m: Wiadomość jawna (liczba całkowita) 

    :param e: Klucz publiczny (eksponent) 

    :param n: Klucz publiczny (moduł) 

    :return: Zaszyfrowana wiadomość (liczba całkowita) 

    """ 

    c = pow(m, e, n)  # c = m^e mod n 

    return c 


def decrypt_message(c, d, n): 

    """ 

    Deszyfrowanie wiadomości. 

    :param c: Zaszyfrowana wiadomość (liczba całkowita) 

    :param d: Klucz prywatny (eksponent) 

    :param n: Klucz prywatny (moduł) 

    :return: Odszyfrowana wiadomość (liczba całkowita) 

    """ 

    m = pow(c, d, n)  # m = c^d mod n 

    return m 


In [4]:

p = generate_prime(1000, 9999) 

q = generate_prime(1000, 9999) 



keys = generate_keys(p, q) 

print("Klucz publiczny:", keys["public_key"]) 

print("Klucz prywatny:", keys["private_key"]) 


# message = "To jest przykładowa wiadomość o długości 50 znaków.".ljust(50)  # Dopasowanie długości do 50 znaków
message = '8'
message_as_int = int.from_bytes(message.encode('utf-8'), byteorder='big')
print("Wiadomość jako liczba całkowita:",  message_as_int)
encrypted_message = encrypt_message(message_as_int, keys["public_key"][0], keys["public_key"][1])
print("Zaszyfrowana wiadomość:", encrypted_message)

decrypted_message = decrypt_message(encrypted_message, keys["private_key"][0], keys["private_key"][1])
print("Odszyfrowana wiadomość:", decrypted_message)

message_decoded = decrypted_message.to_bytes((decrypted_message.bit_length() + 7) // 8, byteorder='big').decode('utf-8')
 
print("Odszyfrowana wiadomość jako tekst:", message_decoded)

Klucz publiczny: (7371197, 20461627)
Klucz prywatny: (287333, 20461627)
Wiadomość jako liczba całkowita: 56
Zaszyfrowana wiadomość: 10647019
Odszyfrowana wiadomość: 56
Odszyfrowana wiadomość jako tekst: 8


---
# 4. Odpowiedzi na pytania. 

### 1. Jakie elementy algorytmu są trudne w realizacji?

- **Generowanie dużych liczb pierwszych**:
  - Znalezienie odpowiednio dużych liczb pierwszych `p` i `q` jest czasochłonne, szczególnie gdy wymagane są liczby o dużej liczbie bitów (np. 1024-bitowe). Sprawdzanie pierwszości dużych liczb wymaga zaawansowanych algorytmów.

- **Wyznaczanie klucza prywatnego `d`**:
  - Obliczenie odwrotności modularnej `e` względem φ(n) wymaga zastosowania rozszerzonego algorytmu Euklidesa. Chociaż Python upraszcza to za pomocą funkcji `pow(e, -1, phi)`, w implementacjach niskopoziomowych jest to bardziej skomplikowane.

- **Zapewnienie względnej pierwszości `e` i φ(n)**:
  - `e` musi być względnie pierwsze z φ(n), co wymaga sprawdzenia za pomocą algorytmu Euklidesa. Dodatkowo, w praktycznych implementacjach `e` musi być odpowiednio dobrane, aby szyfrowanie i deszyfrowanie były efektywne.

- **Ograniczenia rozmiaru wiadomości**:
  - Wiadomość musi być mniejsza niż moduł `n`. W przypadku dłuższych wiadomości konieczne jest dzielenie ich na bloki lub stosowanie paddingu, co komplikuje implementację.

- **Efektywność obliczeń**:
  - Operacje szyfrowania i deszyfrowania dla dużych liczb są kosztowne obliczeniowo, szczególnie dla kluczy o długości 2048 bitów lub większych.

---

### 2. Co stanowi o bezpieczeństwie i jakości tego algorytmu szyfrowania?

- **Trudność faktoryzacji dużych liczb**:
  - Bezpieczeństwo RSA opiera się na problemie faktoryzacji dużych liczb całkowitych. Dla odpowiednio dużych `p` i `q` (np. 1024-bitowych), rozkład `n = p * q` na czynniki pierwsze jest praktycznie niemożliwy w rozsądnym czasie.

- **Rozmiar klucza**:
  - Im większe liczby pierwsze `p` i `q`, tym większy moduł `n`, co zwiększa bezpieczeństwo. W praktyce zaleca się klucze o długości co najmniej 2048 bitów.

- **Wybór `e`**:
  - `e` musi być odpowiednio dobrane, aby zapewnić zarówno bezpieczeństwo, jak i efektywność. Popularnym wyborem jest `e = 65537`, ponieważ jest to liczba pierwsza i umożliwia szybkie obliczenia.

- **Unikalność kluczy**:
  - Każda para kluczy (publiczny i prywatny) jest unikalna dzięki zastosowaniu różnych liczb pierwszych `p` i `q`.

- **Ochrona klucza prywatnego**:
  - Bezpieczeństwo RSA zależy od tajności klucza prywatnego `d`. Jeśli klucz prywatny zostanie ujawniony, cały system szyfrowania staje się podatny na ataki.

- **Padding**:
  - W praktycznych implementacjach stosuje się techniki paddingu (np. OAEP), aby zapobiec atakom na podstawie struktury wiadomości.

- **Odporność na ataki**:
  - RSA jest odporny na wiele rodzajów ataków, takich jak ataki brute force, pod warunkiem, że klucze są odpowiednio duże i stosowane są najlepsze praktyki (np. unikanie małych wartości `e`).

--- 


# Wnioski
